In [1]:
! pip install tokenizer sentencepiece
! nvidia-smi

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/home/geantvert/.local/share/virtualenvs/kernl/bin/python -m pip install --upgrade pip' command.
Sat Oct 15 21:51:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:03:00.0  On |                  N/A |
| 56%   58C    P0    92W / 350W |    139MiB / 24576MiB |     33%      Default |
|                               |      

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import time
import torchdynamo
import torch
from typing import List
from kernl.optimizer.dynamo_backend import dynamo_backend_ofi
from kernl.implementations.cuda_graph import cuda_graphs_wrapper

In [3]:
# default cache size needs to be increased to store the many graphs with generative models
torchdynamo.config.cache_size_limit = 512

model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/geantvert/.local/share/virtualenvs/kernl/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
input_ids = tokenizer(
    "Translate in English: c'est beaucoup plus rapide avec ces optimisations !",
    return_tensors="pt",
    pad_to_multiple_of=8,
    padding=True,
).to("cuda")

In [5]:
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    for _ in range(3):
        output = model.generate(
            inputs=input_ids["input_ids"],
            min_length=100,
            max_length=100,
        )
    start = time.time()
    output = model.generate(
        inputs=input_ids["input_ids"],
        min_length=100,
        max_length=100,
    )
    print(time.time() - start)
    print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))

0.7482590675354004
c'est beaucoup plus rapide avec ces optimisations!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 


In [6]:
model.encoder.forward2 = model.encoder.forward
model.decoder.forward2 = model.decoder.forward

# share cuda pool among all cuda graphs
pool: (int, int) = torch.cuda.graph_pool_handle()


def compiler(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    dynamo_backend_ofi(gm)
    return cuda_graphs_wrapper(gm, example_inputs, pool=pool)


def run_encoder(*args, **kwargs):
    with torchdynamo.optimize(compiler):
        return model.encoder.forward2(*args, **kwargs)


def run_decoder(*args, **kwargs):
    with torchdynamo.optimize(compiler):
        return model.decoder.forward2(*args, **kwargs)

In [7]:
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    out1 = run_decoder(**input_ids)
    out2 = model.decoder(**input_ids)
    assert torch.allclose(out1.last_hidden_state, out2.last_hidden_state, atol=1e-1)
    out3 = run_encoder(**input_ids)
    out4 = model.encoder(**input_ids)
    assert torch.allclose(out3.last_hidden_state, out4.last_hidden_state, atol=1e-1)

In [8]:
model.encoder.forward = run_encoder
model.decoder.forward = run_decoder

In [9]:
# warmup
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    start = time.time()
    model.generate(
        inputs=input_ids["input_ids"],
        min_length=100,
        max_length=100,
    )
    print(time.time() - start)

----------------------------------------
TORCHDYNAMO: backend compiler failed
Traceback (most recent call last):
  File "/home/geantvert/.local/share/virtualenvs/kernl/lib/python3.9/site-packages/torchdynamo/output_graph.py", line 362, in call_user_compiler
    compiled_fn = self.compiler_fn(gm, self.example_inputs())
  File "/tmp/ipykernel_731904/2186925895.py", line 9, in compiler
    return cuda_graphs_wrapper(gm, example_inputs, pool=pool)
  File "/home/geantvert/workspace/kernl/src/kernl/implementations/cuda_graph.py", line 40, in cuda_graphs_wrapper
    model(*inputs)
  File "/home/geantvert/.local/share/virtualenvs/kernl/lib/python3.9/site-packages/torch/fx/graph_module.py", line 652, in call_wrapped
    return self._wrapped_call(self, *args, **kwargs)
  File "/home/geantvert/.local/share/virtualenvs/kernl/lib/python3.9/site-packages/torch/fx/graph_module.py", line 277, in __call__
    raise e
  File "/home/geantvert/.local/share/virtualenvs/kernl/lib/python3.9/site-packages/tor

BackendCompilerFailed: compiler raised AssertionError: Last size of mask must be seq_length to broadcast on QK^t: 24 != 1

You can suppress this exception and fall back to eager by setting:
    torchdynamo.config.raise_on_backend_error = False

In [ ]:
with torch.inference_mode(), torch.autocast(dtype=torch.float16, cache_enabled=True, device_type="cuda"):
    start = time.time()
    output = model.generate(
        inputs=input_ids["input_ids"],
        min_length=100,
        max_length=100,
    )
    print(time.time() - start)
    print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))